# KeplerMapper visualization

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import io

col_names = ["date", "views", "tags", "title", "text"]
with open("956-1010.csv", "r", encoding="utf-8") as data_file:
    df = pd.read_csv(data_file, delimiter=";", names=col_names)

In [2]:
texts = df[df["tags"].str.contains("происшествия") == True]
text_1 = texts.head(10).text.values[8].replace(u'\xa0', u' ')
text_1

'По информации Лайфа, десять москвичей получили тяжёлые травмы из-за обморожений и переохлаждения и были доставлены в больницы в Москве.Девять человек были госпитализированы в столице с обморожениями разной степени, а ещё один — с переохлаждением.— Четверо пострадавших были доставлены в больницы из Центрального административного округа Москвы, ещё по двое пострадавших оказались на севере и на западе столицы, — рассказал источник Лайфа, — также по одному пострадавшему от морозов приходится на ВАО И СЗАО.Одна из госпитализированных — 12-летняя девочка, она поступила в больницу с Крымского вала с обморожением пальцев.'

In [3]:
texts = df[df["tags"].str.contains("технологии") == True]
text_2 = texts.head(10).text.values[2].replace(u'\xa0', u' ')
text_2

'Представленный аксессуар крепится к верхней части смартфона или планшета на платформе ОС Android и достаточно компактен для того, чтобы носить его с собой. Устройство имеет два инфракрасных датчика, датчик цвета и два инфракрасных лазерных прожектора, которые применяются для трёхмерного сканирования объекта.    Для создания 3D-модели своего лица пользователю достаточно просто медленно повернуть голову слева направо. Весь процесс занимает около 20 секунд. Результат сохраняется в стандартном формате .obj, что позволяет использовать полученную модель в других проектах или даже распечатать её на 3D-принтере.    Как отмечается, в настоящий момент устройство находится в стадии разработки, однако первые коммерческие модели могут появиться на рынке уже в конце 2017 года по цене около $300 (примерно 18 тыс. руб.).'

In [4]:
test_text = text_2

In [5]:
import argparse
from elmo_helpers import *
from smart_open import open
import numpy as np
import tensorflow as tf
import nltk
nltk.download('punkt')

def parse(text, elmo_path):
    data_path = input
    sentences = nltk.sent_tokenize(text)
    sentences = [tokenize(s) for s in sentences]

    print('=====')
    print('%d sentences total' % len(sentences))
    print('=====')

    batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings(elmo_path)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        elmo_vectors = get_elmo_vectors(
            sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)

    print('ELMo embeddings are ready')
    print('Tensor shape:', elmo_vectors.shape)

    cropped_vectors = []
    for vect, sent in zip(elmo_vectors, sentences):
        cropped_vector = vect[:len(sent), :]
        cropped_vectors.append(cropped_vector)

    return cropped_vectors, sentences


C:\Users\ariel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ariel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ariel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ariel\AppData\Roaming\Python\Python36\site-

In [6]:
vec, original_data = parse(test_text, "195")

W1126 17:17:56.960733  4396 deprecation_wrapper.py:119] From F:\Study\NIS_Krylov\simple_elmo\elmo_helpers.py:56: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1126 17:17:56.971708  4396 deprecation_wrapper.py:119] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:276: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1126 17:17:56.972706  4396 deprecation_wrapper.py:119] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:333: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1126 17:17:56.986665  4396 deprecation_wrapper.py:119] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:378: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



=====
8 sentences total
=====


W1126 17:17:57.681800  4396 deprecation.py:323] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:522: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W1126 17:17:57.692748  4396 deprecation_wrapper.py:119] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:566: The name tf.nn.rnn_cell.LSTMStateTuple is deprecated. Please use tf.compat.v1.nn.rnn_cell.LSTMStateTuple instead.

W1126 17:17:57.692748  4396 deprecation.py:323] From F:\Study\NIS_Krylov\simple_elmo\bilm\model.py:567: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W1126 17:17:57.739655  4396 deprecation.py:506] From C:\Users\ariel\AppData\Roaming\Python\Python36\site-packages\tensorflow

ELMo embeddings are ready
Tensor shape: (8, 30, 1024)


In [7]:
stacked_vec = np.vstack(vec)
print(stacked_vec.shape)

(113, 1024)


In [8]:
original_text = np.hstack(original_data)
len(original_text)

113

In [9]:
import kmapper as km
from sklearn.manifold import Isomap
from sklearn.preprocessing import MinMaxScaler
mapper = km.KeplerMapper(verbose=2)

projected_X = mapper.fit_transform(stacked_vec,
    projection=[Isomap(n_components=2, n_jobs=-1)],
    scaler=[None, None, MinMaxScaler()])

print("SHAPE",projected_X.shape)

KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: Isomap(eigen_solver='auto', max_iter=None, n_components=2, n_jobs=-1,
       n_neighbors=5, neighbors_algorithm='auto', path_method='auto', tol=0)
	Distance matrices: False
	Scalers: None
..Projecting on data shaped (113, 1024)

..Projecting data using: 
	Isomap(eigen_solver='auto', max_iter=None, n_components=2, n_jobs=-1,
       n_neighbors=5, neighbors_algorithm='auto', path_method='auto', tol=0)

SHAPE (113, 2)


In [10]:
from sklearn import cluster
graph = mapper.map(projected_X,
                   clusterer=cluster.AgglomerativeClustering(n_clusters=3,
                                                             linkage="complete",
                                                             affinity="cosine"),
                   overlap_perc=0.47)

Mapping on data shaped (113, 2) using lens shaped (113, 2)

Minimal points in hypercube before clustering: 3
Creating 100 hypercubes.
Cube_0 is empty.

   > Found 3 clusters.

Cube_2 is empty.

   > Found 3 clusters.

   > Found 3 clusters.

Cube_5 is empty.

Cube_6 is empty.

Cube_7 is empty.

   > Found 3 clusters.

Cube_9 is empty.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

Cube_17 is empty.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

Cube_26 is empty.

Cube_27 is empty.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

   > Found 3 clusters.

Cube_34 is empty.

Cube_35 is empty.

Cube_36 is empty.

   > Found 3 clusters.

   > Found 3 cluster

In [11]:
html = mapper.visualize(graph,
                        lens=projected_X,
                        lens_names=["ISOMAP1", "ISOMAP2"],
                        path_html="технологии.html",
                        X=stacked_vec,
                        X_names=original_text,
                        title="технологии",
                       )

Wrote visualization to: технологии.html
